# Imports

In [1]:
import numpy as np
from pprint import pprint as pp
import re #Used to read chunks from the data files - clusters, vmat, etec.
import math
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
import random
from IPython.display import clear_output
import itertools
import sys

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
import plotly.figure_factory as ff
import matplotlib.tri as tri

from scipy.spatial import Delaunay
from scipy.optimize import minimize, basinhopping
from scipy.optimize import SR1, BFGS
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint

from ase import Atoms
from ase.units import kB
from ase.visualize import view
from ase.build import bulk
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.vasp import write_vasp

import sys
import subprocess
import os

pattern1 = re.compile("\n\n\n")
pattern2 = re.compile("\n\n")
np.set_printoptions(suppress=True,precision=2)
random.seed(42)
np.random.seed(seed=42)

# Read all necessary files

## Read clusters.out

In [2]:
# Read clusters.out
clusters = {}

with open('clusters.out','r') as fclusters:
    temp_clusters = fclusters.read().split('\n\n') #Read blocks separated by 1 empty line

for idx, cluster in enumerate(temp_clusters):
    if cluster == '': #Check for spurious empty blocks
        continue 
    line = cluster.split('\n') #If not empty split by lines
    multiplicity = int(line[0]) #1st line
    length = float(line[1]) #largest distance between two atoms
    num_points = int(line[2]) #type of cluster
    clusters[idx] = {'mult':multiplicity, 'length':length, 'type':num_points}

In [3]:
clusters

{0: {'mult': 1, 'length': 0.0, 'type': 0},
 1: {'mult': 1, 'length': 0.0, 'type': 1},
 2: {'mult': 4, 'length': 0.86603, 'type': 2},
 3: {'mult': 3, 'length': 1.0, 'type': 2},
 4: {'mult': 12, 'length': 1.0, 'type': 3},
 5: {'mult': 6, 'length': 1.0, 'type': 4}}

## Read config.out

In [4]:
# Read config.out
configs = {}

fconfig = open('config.out','r')
_ = next(fconfig) #Ignore first line

temp_config = fconfig.read()#.split('\n\n')
temp_config = pattern1.split(temp_config) #split lines separated by 2 empty lines

for idx, config in enumerate(temp_config):
    if config == '': #Check for spurious empty blocks
        continue
    num_points = int(config[0]) #number of subclusters
    config = pattern2.split(config[2:]) #now split individual subclusters separated by 1 blank line
    min_coords = []
    for _ in range(num_points):
        min_coords.append(config[_].split('\n')[0])
    configs[idx] = {'subclus': list(map(int,min_coords)), 'num_of_subclus': len(min_coords)}

## Read Kikuchi-Baker coefficients

In [5]:
# Read kb.out

kb = {}
fkb = open('kb.out','r')
_ = next(fkb) #ignore first line

temp_kb = fkb.read()
temp_kb = temp_kb.split('\n') #split file linewise

for idx, kbcoeff in enumerate(temp_kb):
    if kbcoeff == '': #check for spurious empty blocks
        continue
    kb[idx] = float(kbcoeff)

fkb.close()

## Read coefficients of subclusters

In [6]:
# Read configcoeff.out
configcoef = {}

with open('configcoef.out','r') as fsubmult:
    _ = next(fsubmult) #ignore first line
    temp_submult = fsubmult.read() 
    temp_submult = pattern2.split(temp_submult) #split lines into blocks separated by 2 empty lines
    
for idx, submult in enumerate(temp_submult):
    submult = submult.split('\n') #split into number of subclusters 
    while("" in submult) :
        submult.remove("") #remove empty blocks
    configcoef[idx] = list(map(float,submult[1:])) #also ignore 1st line of each block

In [7]:
configcoef

{0: [1.0],
 1: [1.0, 1.0],
 2: [1.0, 2.0, 1.0],
 3: [1.0, 2.0, 1.0],
 4: [1.0, 2.0, 1.0, 1.0, 2.0, 1.0],
 5: [1.0, 4.0, 2.0, 4.0, 4.0, 1.0],
 6: [0.0, -1.0, 1.0, 1.0, -1.0, 1.0]}

## Read V-Matrix

In [8]:
# Read vmat.out
vmat = {}
with open('vmat.out') as fvmat:
    _ = next(fvmat) #ignore first lie
    temp_vmat = fvmat.read()
    temp_vmat = pattern2.split(temp_vmat) #split by 2 empty lines i.e. maxclusters
    
    while("" in temp_vmat):
        temp_vmat.remove("") #remove empty blocks
    
    for clus_idx, mat in enumerate(temp_vmat):
        mat = mat.split('\n') #split by 1 empty line i.e. subclusters
        mat_float = np.empty(list(map(int, mat[0].split(' '))))
        for idx, row in enumerate(mat[1:]): #ignore first line
            mat_float[idx] = list(map(float,row.split(' ')[:-1]))
        
        vmat[clus_idx] = mat_float

In [9]:
vmat

{0: array([[1., 0., 0., 0., 0., 0.]]),
 1: array([[ 0.5, -0.5,  0. ,  0. ,  0. ,  0. ],
        [ 0.5,  0.5,  0. ,  0. ,  0. ,  0. ]]),
 2: array([[ 0.25, -0.5 ,  0.25,  0.  ,  0.  ,  0.  ],
        [ 0.25,  0.  , -0.25,  0.  ,  0.  ,  0.  ],
        [ 0.25,  0.5 ,  0.25,  0.  ,  0.  ,  0.  ]]),
 3: array([[ 0.25, -0.5 ,  0.  ,  0.25,  0.  ,  0.  ],
        [ 0.25,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
        [ 0.25,  0.5 ,  0.  ,  0.25,  0.  ,  0.  ]]),
 4: array([[ 0.12, -0.38,  0.25,  0.12, -0.12,  0.  ],
        [ 0.12, -0.12,  0.  , -0.12,  0.12,  0.  ],
        [ 0.12,  0.12, -0.25,  0.12, -0.12,  0.  ],
        [ 0.12, -0.12, -0.25,  0.12,  0.12,  0.  ],
        [ 0.12,  0.12,  0.  , -0.12, -0.12,  0.  ],
        [ 0.12,  0.38,  0.25,  0.12,  0.12,  0.  ]]),
 5: array([[ 0.06, -0.25,  0.25,  0.12, -0.25,  0.06],
        [ 0.06, -0.12,  0.  ,  0.  ,  0.12, -0.06],
        [ 0.06,  0.  , -0.25,  0.12,  0.  ,  0.06],
        [ 0.06,  0.  ,  0.  , -0.12,  0.  ,  0.06],
        [ 0.06

## Read ECI

In [10]:
# Read eci
eci = {}

with open('eci.out') as feci:
    _ = next(feci) #Ignore first line
    temp_eci = feci.read()
    temp_eci = temp_eci.split('\n') #split by line

for idx, eci_val in enumerate(temp_eci):
    if eci_val == '':
        continue
    eci[idx] = float(eci_val)

In [11]:
#In this notebook ECI's are declared manually
eci_2 = 0.01
eci_3 = 0.05*eci_2
eci = {0: 0, 1: 0.0, 2: eci_2, 3: eci_3, 4: 0, 5: 0}

In [12]:
eci

{0: 0, 1: 0.0, 2: 0.01, 3: 0.0005, 4: 0, 5: 0}

In [13]:
corrs1 = np.array([1., 1., 1., 1., 1., 1.]) #Pure B
corrs0 = np.array([1., -1., 1., 1., -1., 1.]) #Pure A
corrssqs = np.array([1.    , 0.0   , 0.25,0.25  , 0.125 , 0.0625]) # AB - sqs
corrsrand = np.array([1.   , *np.random.uniform(-1, 1, 5)]) 
T = 1/kB
print(corrsrand)

[ 1.   -0.25  0.9   0.46  0.2  -0.69]


In [14]:
for cluster_idx, _ in clusters.items():
    rho = np.matmul(vmat[cluster_idx],corrssqs)
    print(rho)

[1.]
[0.5 0.5]
[0.31 0.19 0.31]
[0.31 0.19 0.31]
[0.2  0.11 0.08 0.11 0.08 0.23]
[0.13 0.07 0.04 0.04 0.04 0.19]


# Setting up Log Absolute functions

## Set up F

In [15]:
def F(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    F = H + kB*T*SUM(rho * log(rho))
    """
    
    def get_corrsum(vmat,corrs):
        assert len(vmat) == len(corrs)
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return 0
            #corrsum = np.finfo(float).tiny

        return corrsum * math.log(np.abs(corrsum))
    
    def per_cluster_sum(corrs,vmat,configcoef):
        config_sum = np.sum([coef * get_corrsum(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)
                            ])
                      
        return config_sum
    
    H = np.sum([cluster['mult']*eci[cluster_idx]*corrs[cluster_idx] 
                for cluster_idx, cluster in clusters.items()
               ])
    
    S = np.sum([kb[cluster_idx]*per_cluster_sum(corrs,
                                                vmat[cluster_idx],
                                                configcoef[cluster_idx],)
                for cluster_idx in clusters.keys()
               ])
    
    return H + kB*T*S

In [16]:
f0 = F(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f1 = F(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
fsqs = F(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
frand = F(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f0:.2f} -- Corrs 1: {f1:.2f} -- Corrs Rand: {frand:.2f} -- Corrs SQS: {fsqs:.2f}")

Corrs 0: 0.04 -- Corrs 1: 0.04 -- Corrs Rand: -1.65 -- Corrs SQS: -2.62


## Set up F Jacobian

In [17]:
def F_jacobian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [dF/dcorr0, dF/dcorr1, ...]
    """
    
    def get_kth_elem_jac(corrs, vmat, kb, clusters, configs, configcoef,corr_idx):
        
        dS_k = np.sum([kb[cluster_idx]*per_cluster_sum_jac(corrs,
                                                           vmat[cluster_idx],
                                                           configcoef[cluster_idx],
                                                           corr_idx,) 
                       for cluster_idx in clusters.keys()
                      ])
        return dS_k
    
    def get_corrsum_jac(vmat,corrs):
        assert len(vmat) == len(corrs)
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return np.NINF
            #corrsum = np.finfo(float).tiny

        return 1 + math.log(np.abs(corrsum))

    def per_cluster_sum_jac(corrs,vmat,configcoef,corr_idx):
        
        config_sum = np.sum([coef * vmat[config_idx][corr_idx] * get_corrsum_jac(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)
                            ])

        
        return config_sum
    
    dH = np.array([cluster['mult']*eci[cluster_idx] for cluster_idx, cluster in clusters.items()])
    
    dS = np.array([get_kth_elem_jac(corrs, vmat, kb, clusters, configs, configcoef,corr_idx) for corr_idx, _ in enumerate(corrs)])
    
    return dH + kB*T*dS

In [18]:
f_jaco0 = F_jacobian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_jaco1 = F_jacobian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_jacosqs = F_jacobian(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
f_jacorand = F_jacobian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f_jaco0} \nCorrs 1: {f_jaco1} \nCorrs Rand {f_jacorand} \nCorrs SQS {f_jacosqs}")

Corrs 0: [nan nan nan nan nan nan] 
Corrs 1: [nan nan nan nan nan nan] 
Corrs Rand [-1.76 -0.5   0.66  0.73  0.25  0.  ] 
Corrs SQS [-1.9  -0.1   0.61  0.42  0.23 -0.05]


/home/sayan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-17-873df088579e>:38: RuntimeWarning: invalid value encountered in double_scalars
  config_sum = np.sum([coef * vmat[config_idx][corr_idx] * get_corrsum_jac(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)


## Set up Hessian

In [19]:
def F_hessian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [[d^2F/dcorr0 dcorr0, d^2F/dcorr0 dcorr1, ..., d^2F/dcorr0 dcorrn],
     [d^2F/dcorr1 dcorr0, d^2F/dcorr1 dcorr1, ..., d^2F/dcorr1 dcorrn],
     .
     .
     .
     [d^2F/dcorrn dcorr0, d^2F/dcorrn dcorr1, ..., d^2F/dcorrn dcorrn],
    ]
    """
    
    def get_corrsum_hess(vmat,corrs):
        assert len(vmat) == len(corrs)
        
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return 1/np.PINF
            #corrsum = np.finfo(float).tiny

        return corrsum
    
    def get_config_val(corrs,vmat,configcoef,corr_idx_1,corr_idx_2):
        
        config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs) 
                             for config_idx, coef in enumerate(configcoef)
                            ])
        
        return config_val
    
    def get_hessian_elem(corrs, vmat, kb, clusters, configs, configcoef,T,eci,corr_idx_1,corr_idx_2):
        
        hess_elem = np.sum([kb[cluster_idx] * get_config_val(corrs,
                                                             vmat[cluster_idx],
                                                             configcoef[cluster_idx],
                                                             corr_idx_1,
                                                             corr_idx_2
                                                            ) 
                            for cluster_idx in clusters.keys()
                           ])
        return hess_elem
    
    d2F = np.empty([len(corrs),len(corrs)])
    
    d2F = np.array([[get_hessian_elem(corrs, vmat, kb, clusters, configs, configcoef, T, eci, corr_idx_1, corr_idx_2) for corr_idx_2, _ in enumerate(corrs)] for corr_idx_1, _ in enumerate(corrs)])
    
    return d2F

In [20]:
f_hess0 = F_hessian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_hess1 = F_hessian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_hessrand = F_hessian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
f_hesssqs = F_hessian(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0:\n {f_hess0} \n Corrs 1:\n {f_hess1}\n Corrs Rand:\n {f_hessrand} \n Corrs SQS:\n {f_hesssqs}")

Corrs 0:
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] 
 Corrs 1:
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
 Corrs Rand:
 [[ 1.42 -0.74 -2.66  2.25  0.85 -0.85]
 [-0.74 -1.12 -2.52  4.13  2.61 -0.43]
 [-2.66 -2.52  0.67  2.44  2.59  0.31]
 [ 2.25  4.13  2.44 -4.58 -3.9   0.76]
 [ 0.85  2.61  2.59 -3.9  -2.1   0.44]
 [-0.85 -0.43  0.31  0.76  0.44 -0.02]] 
 Corrs SQS:
 [[ 1.25  0.13 -0.55 -0.41 -0.16  0.14]
 [ 0.13  2.99  0.02  0.01 -0.89 -0.35]
 [-0.55  0.02  3.49 -0.97 -0.25 -0.69]
 [-0.41  0.01 -0.97  2.76 -0.13 -0.34]
 [-0.16 -0.89 -0.25 -0.13  1.94  0.27]
 [ 0.14 -0.35 -0.69 -0.34  0.27  1.29]]


<ipython-input-19-d4a292fd34bd>:35: RuntimeWarning: divide by zero encountered in double_scalars
  config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs)
<ipython-input-19-d4a292fd34bd>:35: RuntimeWarning: invalid value encountered in double_scalars
  config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs)


# Constraints

In [21]:
def constraint_rhos_sum(corrs, vmat, clusters, configcoef,):
    """
    Constraints the sum of each rho. As of now, it's done in a weird way, where the total sum of the array:
    [1 - sum(rho), .... ] is constrained to sum to 0. This along with the constraint that each rho is between
    0 and 1, seems to make it work. I think that by the this might be a redundant constraint as well.
    """
    rho_sum = []

    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        rho_sum.append(np.sum(configcoef[cluster_idx]*rho))
    
    return np.sum(1 - np.array(rho_sum))

def constraint_singlet(corrs,FIXED_CORR_1):
    """
    constrains the 1-pt correlation:
    corrs[1] = FIXED_CORR_1
    """
    return corrs[1] - FIXED_CORR_1   

def constraint_zero(corrs):
    """
    constrains the 1-pt correlation:
    corrs[0] = 1
    """
    return 1 - corrs[0]

def constraint_NN(corrs,FIXED_CORR_2):
    """
    constrains the 2-pt correlation:
    corrs[2] = FIXED_CORR_2
    """
    return corrs[2] - FIXED_CORR_2 

class MyBounds:
    """
    Class to constrain the trial correlations of Basin Hopping
    """
    def __init__(self,xmax=[1]*6, xmin=[-1]*6,):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))

        return tmax and tmin
    
class MyTakeStep:
    
    def __init__(self, vmat, clusters, stepsize=0.1):
        self.stepsize = stepsize
        self.vmat = vmat
        self.clusters = clusters
        self.rng = np.random.default_rng()
    
    def __call__(self, x):
        s = self.stepsize
        
        validcorr = np.ones(len(self.clusters), dtype=bool)
        
        for _ in iter(int,1):
            x_trial = x + self.rng.uniform(-s, s, x.shape)
            for cluster_idx, _ in self.clusters.items():
                rho = np.matmul(self.vmat[cluster_idx],x_trial)
                validcorr[cluster_idx] = np.all(rho >= 0)
            if bool(np.all(validcorr)):
                break
            
        return x_trial

# Uniform Sampling

In [ ]:
np.linspace(0, 500, num=11)

In [ ]:
results_uniform = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])
NUM_TRIALS = 100
MAX_TEMP = 500

for temp in tqdm(np.linspace(0, MAX_TEMP, num=11)):
    for x in [0]:#tqdm(np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,9)):
        FIXED_CORR_1 = x
        
        MIN_RES_VAL = 1e5 #random large number
        rho_pair = np.array([None,None])
        #corrs0 = np.array([1, *np.random.uniform(-1, 1, 2)])
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():
            
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)]
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 3000,
                   'xtol'    : 1e-15,
                   'initial_constr_penalty' : 10,
                  }
        
        for _ in tqdm(range(NUM_TRIALS)):
            
            for _ in iter(int,1): #infinite loop till a valid starting correlations are found
                corrs0 = np.array([1, x, *np.random.uniform(-1, 1, len(clusters)-2)])
                validcorr = np.ones(len(clusters), dtype=bool)
        
                for cluster_idx, _ in clusters.items():
                    rho = np.matmul(vmat[cluster_idx],corrs0)
                    validcorr[cluster_idx] = np.all(rho >= 0)
                
                if bool(np.all(validcorr)):
                    break   
                
            res = minimize(F,
                           corrs0,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci),
                           options=options,
                           #jac='3-point',
                           #hess=BFGS(),
                           jac=F_jacobian,
                           hess=F_hessian,
                           constraints=[#*linear_constraints, 
                                        {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                        {'fun': constraint_zero, 'type':'eq',},
                                       ],
                           bounds=bounds_corrs,
                          )
            
            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for x:{x}, T:{temp} fun: {MIN_RES_VAL}")
                print(f'Current minimum correlations: {res.x}')
                #for cluster_idx in clusters.keys():
                #    print(np.matmul(vmat[cluster_idx],res.x))
        
            
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_uniform = results_uniform.append({'T' : temp, 
                                           '1-point_corr' : x, 
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                          }, 
                                          ignore_index = True
                                         )
        
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
results_uniform.to_pickle(f'results/uni_{eci[2]},{eci[3]}_{MAX_TEMP}.pickle')

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  warn('Singular Jacobian matrix. Using SVD decomposition to ' +
/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Found new minimum for x:0, T:0.0 fun: -0.02570492425917527
Current minimum correlations: [ 1.   -0.   -0.67  0.64  0.    0.38]
Found new minimum for x:0, T:0.0 fun: -0.040919013325830034
Current minimum correlations: [ 1.   -0.   -1.   -0.7   0.   -0.14]
Found new minimum for x:0, T:0.0 fun: -0.0412812237336417
Current minimum correlations: [ 1.    0.   -1.   -0.91 -0.   -0.12]
Found new minimum for x:0, T:0.0 fun: -0.041352589431319406
Current minimum correlations: [ 1.   -0.   -1.   -0.95  0.1   0.6 ]
Found new minimum for x:0, T:0.0 fun: -0.041461920823634146
Current minimum correlations: [ 1.   -0.   -1.   -0.99  0.    0.2 ]
Found new minimum for x:0, T:0.0 fun: -0.041466498792397034
Current minimum correlations: [ 1.   -0.   -1.   -0.99  0.   -0.14]
Found new minimum for x:0, T:0.0 fun: -0.04147095067225223
Current minimum correlations: [ 1.   -0.   -1.   -0.99  0.    0.47]
Found new minimum for x:0, T:0.0 fun: -0.041490942573939975
Current minimum correlations: [ 1.    0.   -1.  

In [ ]:
eci

## Plot Phase Diagram

In [ ]:
fig = go.Figure()

for T in results_uniform['T'].unique():
    fig.add_trace(go.Scatter(x = results_uniform[results_uniform['T'] == T]['1-point_corr'],
                             y = results_uniform[results_uniform['T'] == T]['F'],
                             mode='markers+lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

## Plot Pair Correlation

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = results_uniform[results_uniform['1-point_corr'] == 0.0]['T'],
                         y = results_uniform[results_uniform['1-point_corr'] == 0.0]['corrs'].str[2],
                         mode='markers+lines',
                        )
             )

fig.update_layout(
    title="T vs 2-point Corr",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

In [ ]:
np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,19)

# Basin Hopping

## Defining the Callback

In [ ]:
def basin_hopping_callback(corrs, G, accept):
    """
    Function to print diagnostic while fitting. 
    Not being used right now
    """
    if accept == True:
        clear_output(wait=True)
        print(f'1-pt Correlation: {corrs[1]:.2f}')
        print(f'Concentation: {(corrs[1] - (-1))/(1 - (-1)):.2f}')
        print(f'New Minima found --> G: {G:.2f}')
        print('Current Rho:')
        for cluster_idx in clusters:
            print(np.matmul(vmat[cluster_idx],corrs))
        print("===========================")
        
        

## Optimisation Code

In [ ]:
results_basinhopping = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])
MAX_TEMP = 500
now = datetime.now()
now = now.strftime("%d-%m-%H-%M")

for temp in tqdm(np.linspace(0, MAX_TEMP, num=21)):
    for x in [0]:#tqdm(np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,9)):

        FIXED_CORR_1 = x

        rho_pair = np.array([None,None])
        
        #Make a first guess
        for _ in iter(int,1): #infinite loop till a valid starting correlations are found
            corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
            validcorr = np.ones(len(clusters), dtype=bool)

            for cluster_idx, _ in clusters.items():
                rho = np.matmul(vmat[cluster_idx],corrs0)
                validcorr[cluster_idx] = np.all(rho >= 0)

            if bool(np.all(validcorr)):
                break   
        
        #corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
        linear_constraints = []

        #Linear Constraint for each rho to be between 0 and 1
        for cluster_idx, _ in clusters.items():
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        #Set bounds for the local minimisation 
        #limit correlations to [1, 1-pt, [-1,1], [-1,1], ...]
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)],
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 5000,
                   'xtol'    : 1e-9,
                   'initial_constr_penalty' : 10,
                  }
        
        minimizer_kwargs = {'args':(vmat, kb, clusters, configs, configcoef,temp,eci),
                            'method': 'trust-constr',
                            'options': options,
                            'jac': F_jacobian, 'hess': F_hessian,
                            'constraints' : [*linear_constraints, 
                                             {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1], 'hess': 0},
                                             {'fun': constraint_zero, 'type':'eq','hess': 0},
                                             #{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,]},
                                            ],
                            'bounds': bounds_corrs,
                           }
        
        #Bounds for trial correlations
        mybounds = MyBounds(xmax=[1, FIXED_CORR_1,*[1]*(len(clusters)-2)], 
                            xmin=[1, FIXED_CORR_1,*[-1]*(len(clusters)-2)]
                           )
        
        mytakestep = MyTakeStep(vmat,
                                clusters,
                                stepsize=0.05
                               )
        
        res = basinhopping(F, 
                           corrs0, #first guess
                           niter=1000, #total num of iterations
                           T=0.01, #temp for Metropolis MC trial search
                           #stepsize=0.1,
                           minimizer_kwargs=minimizer_kwargs,
                           niter_success=10, #num iters to exit after no new minima found 
                           interval=5, #num iters to change step size
                           disp=True,
                           #accept_test=mybounds,
                           take_step=mytakestep,
                           #seed=42,
                           #callback=basin_hopping_callback
                          )

        #Code to extract rhos and check if they sum them to 1 for sanity. Not used.
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_basinhopping = results_basinhopping.append({'T' : temp, 
                                                     '1-point_corr' : x, 
                                                     'F' : res.fun, 
                                                     'corrs': res.x,
                                                    }, 
                                                    ignore_index = True
                                                   )

#save results
results_basinhopping.to_pickle(f'results/bh_{eci[2]}_{MAX_TEMP}.pickle')

## Plot Phase Diagram

In [ ]:
fig = go.Figure()

for T in results_basinhopping['T'].unique():
    fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['T'] == T]['1-point_corr'],
                             y = results_basinhopping[results_basinhopping['T'] == T]['F'],
                             mode='lines+markers',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

## Plot Pair Correlation



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['T'],
                         y = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['corrs'].str[2],
                         mode='markers+lines',
                        )
             )

fig.update_layout(
    title="T vs 2-point Corr",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

# Fix 1-point and NN

In [ ]:
def singlet_doublet_validpts(triangle = np.array([
    [0, 0],
    [-1, 1],
    [1, 1],
])):
    
    def uniform_triangle(u, v):
        while True:
            s = random.random()
            t = random.random()
            in_triangle = s + t <= 1
            p = s * u + t * v if in_triangle else (1 - s) * u + (1 - t) * v
            yield p

    it = uniform_triangle(
        triangle[1] - triangle[0],
        triangle[2] - triangle[0],
    )

    points = np.array(list(itertools.islice(it, 0, 100)))
    points += triangle[0]
    
    return points

In [ ]:
def get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=0, x2=-1, y2=1, x3=1, y3=1, num = 10):
    
    def isInside(x, y, x1=0, y1=0, x2=-1, y2=1, x3=1, y3=1,):
    
        def area(x1, y1, x2, y2, x3, y3):
            return abs((x1*(y2 - y3) + x2*(y3 - y1) + x3*(y1 - y2))/2.0)

        # Calculate area of triangle ABC
        A = area(x1, y1, x2, y2, x3, y3)
        # Calculate area of triangle PBC
        A1 = area(x, y, x2, y2, x3, y3)
        # Calculate area of triangle PAC
        A2 = area(x1, y1, x, y, x3, y3)
        # Calculate area of triangle PAB
        A3 = area(x1, y1, x2, y2, x, y)
        # Check if sum of A1, A2 and A3
        # is same as A
        if(A == A1 + A2 + A3):
            return True
        else:
            return False
                                   

    points = []
    #xx, yy = np.meshgrid(xvalues, yvalues)
    grid = np.meshgrid(xvalues, yvalues)
    grid = np.vstack(list(map(np.ravel, grid))).T

    for x, y in grid:
        if isInside(x,y,x1=x1,y1=y1,x2=x2,y2=y2,x3=x3,y3=y3):
            points.append([x,y])
    
    return np.array(points)

In [ ]:
#singlet_doublet_pairs = singlet_doublet_validpts()
num = 8
xvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
yvalues = np.linspace(0,1,num)
singlet_doublet_pairs_1 = get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=0, x2=-1, y2=1, x3=1, y3=1, num = num)

xvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
yvalues = np.linspace(0,-1,num)
singlet_doublet_pairs_2 = get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=0, x2=-1, y2=-1, x3=1, y3=-1,num=num)

xvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
yvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
singlet_doublet_pairs_3 = get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=-1, x2=-1, y2=1, x3=1, y3=1,num=num)
singlet_doublet_pairs_3 = np.append(singlet_doublet_pairs_3,[[0,-1+np.finfo(float).eps]],axis=0)

singlet_doublet_pairs = np.vstack([np.array(singlet_doublet_pairs_1),np.array(singlet_doublet_pairs_2)])
plt.style.use('default')
plt.scatter(singlet_doublet_pairs_3[:, 0], singlet_doublet_pairs_3[:, 1], s=5)
print(len(singlet_doublet_pairs_3))
plt.show()

In [ ]:
results_blanket = pd.DataFrame(columns = ['T', '1-point_corr', '2-point_corr', 'F','corrs'])
NUM_TRIALS = 50
MAX_TEMP = 500

#corrsrange = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,21)
#singlet_doublet_pairs = singlet_doublet_validpts()
#singlet_doublet_pairs = get_uniform_traingular_points(num=20)
print(f'Working ECIs: {eci}')
for temp in tqdm(np.linspace(0, MAX_TEMP, num=11)):
    for FIXED_CORR_1, FIXED_CORR_2 in tqdm(singlet_doublet_pairs_3):
        
        print('\n=============================================================')
        print(f'Working on 1-point corr: {FIXED_CORR_1:.4f}, NN corr: {FIXED_CORR_2:.4f}, Temp: {temp}')
        print('=============================================================\n')

        MIN_RES_VAL = 1e5 #random large number
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():

            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))

        bounds_corrs = Bounds([1, FIXED_CORR_1, FIXED_CORR_2, *[-1]*(len(clusters)-3)],
                              [1, FIXED_CORR_1, FIXED_CORR_2, *[1]*(len(clusters)-3)]
                             )

        options = {'verbose' : 0,
                   'maxiter' : 3000,
                   'xtol'    : 1e-15,
                   'initial_constr_penalty' : 10,
                  }

        for _ in tqdm(range(NUM_TRIALS)):

            corrs0 = np.array([1, FIXED_CORR_1, FIXED_CORR_2, *np.random.uniform(-1, 1, len(clusters)-3)])

            res = minimize(F,
                           corrs0,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci),
                           options=options,
                           #jac='3-point',
                           #hess=BFGS(),
                           jac=F_jacobian,
                           hess=F_hessian,
                           constraints=[*linear_constraints, 
                                        {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                        {'fun': constraint_zero, 'type':'eq',},
                                        {'fun': constraint_NN, 'type':'eq','args':[FIXED_CORR_2]}
                                       ],
                           bounds=bounds_corrs,
                          )

            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for Corr1:{FIXED_CORR_1:.4f}, Corr2:{FIXED_CORR_2:.4f} fun: {MIN_RES_VAL:.15f}")
                print(f'Current minimum correlations: {res.x}')
#                     for cluster_idx, _ in clusters.items():
#                         rho = np.matmul(vmat[cluster_idx],res.x)
#                         print(rho)


        break_next = False
        for cluster_idx in clusters.keys():
            try:
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
            except AssertionError:
                break_next = True
        if break_next:
            print('No valid solution found')
            continue

        results_blanket = results_blanket.append({'T' : temp, 
                                           '1-point_corr' : FIXED_CORR_1,
                                           '2-point_corr' : FIXED_CORR_2,
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                          }, 
                                          ignore_index = True
                                         )
        #print(results_blanket)
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
results_blanket.to_pickle(f'results/blanket_{eci[2]}_{eci[3]}_{MAX_TEMP}.pickle')
results_blanket.to_pickle('temp.pickle')

In [ ]:
now = datetime.now()
now = now.strftime("%d")
results_blanket.to_pickle(f'results/blanket_{eci[2]}_{MAX_TEMP}_{now}.pickle')

In [ ]:
results_blanket['T'].unique()

In [ ]:
temp = 1000
results_blanket[
                (results_blanket['T'] == temp)
               ]

Xs = results_blanket[results_blanket['T'] == temp]['1-point_corr'].values
Ys = results_blanket[results_blanket['T'] == temp]['2-point_corr'].values
Zs = results_blanket[results_blanket['T'] == temp]['F'].values

fig, ax2 = plt.subplots(nrows=1,figsize=(8, 6), dpi=100)
plt.style.use('ggplot')
ax2.set_title(f'T = {temp}')
ax2.tricontour(Xs, Ys, Zs, levels=10, linewidths=0.5,)
cntr2 = ax2.tricontourf(Xs, Ys, Zs, levels=14,)

fig.colorbar(cntr2, ax=ax2,label='F')
ax2.plot(Xs, Ys, 'ko', ms=3)
ax2.set(xlim=(-1, 1), ylim=(-1, 1))

plt.xlabel("1-point Correlation")
plt.ylabel("2-point NN Correlation")
plt.subplots_adjust(hspace=0.5)

plt.show()

In [ ]:
Xs

In [ ]:
results_blanket['T'].unique()

In [ ]:
fig = go.Figure()
#fig.add_trace(traces)
fig.update_layout(
    xaxis_title="1-point Corr",
    yaxis_title="2-point Corr",
    template='ggplot2',
    legend_title="Legend Title",
)
for temp in [0,150,200,300,450]:#[0.,  50., 100., 150., 200., 250., 300., 350., 400., 450., 500.]:
    Xs = results_blanket[results_blanket['T'] == temp]['1-point_corr'].values
    Ys = results_blanket[results_blanket['T'] == temp]['2-point_corr'].values
    Zs = results_blanket[results_blanket['T'] == temp]['F'].values
    points2D = np.vstack([Xs,Ys]).T
    tri = Delaunay(points2D)

    simplices = tri.simplices
    temp_trace = ff.create_trisurf(x=Xs, y=Ys, z=Zs,
                                   simplices=simplices,
                                   title="F vs singlet-NN-corrlations",
                                   show_colorbar=False,
                                   width=1000,
                                   height=1000,
                                   #aspectratio=dict(x=1, y=1, z=10)
                                  )
    fig.add_traces([temp_trace.data[0]])

fig.layout.scene.xaxis.title='1-point Correlation'
fig.layout.scene.yaxis.title='2-point Correlation'
fig.layout.scene.zaxis.title='F'

fig.update_layout(
    autosize=False,
    width=750,
    height=750,
)


fig.show()

In [ ]:
fig.layout.legend.

In [ ]:
# fig = go.Figure()
# for T in results_blanket['T'].unique():
#     fig.add_trace(go.Scatter(x = results_blanket[results_blanket['T'] == T]['2-point_corr'],
#                              y = results_blanket[results_blanket['T'] == T]['F'],
#                              mode='markers+lines',
#                              name=f"T = {T}: ECI = {results_blanket.iloc[0]['F']/4:.2f}",
#                             )
#                  )
    
# fig.update_layout(
#     title="T vs 2-point Corr",
#     xaxis_title="2-point Corr",
#     yaxis_title="Energy",
#     template='seaborn'
# )
# fig.show()